In [1]:
import pandas as pd
from feature_engine.encoding import MeanEncoder, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [2]:
# let's load the data set

data = pd.read_csv("credit_approval_uci.csv")

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [3]:
# Let's separate into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=["target"], axis=1),  # predictors
    data["target"],  # target
    test_size=0.3,  # percentage of observations in test set
    random_state=0,  # seed to ensure reproducibility
)

X_train.shape, X_test.shape

((483, 15), (207, 15))

In [4]:
# let's select the categorical variables

vars_categorical = X_train.select_dtypes(include="O").columns.to_list()

vars_categorical

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [5]:
# To use K-means we need to compute meaningful distance metrics.
# Thus, we need to encode the categories into values that 
# will provide useful distance metrics.

# This can be achieved through one-hot encoding, 
# frequency encoding and Target encoding.

# let's set up the target encoder

encoder = MeanEncoder(variables=vars_categorical)

# fit the encoder
encoder.fit(X_train, y_train)

# let's transform train and test sets
X_train_enc = encoder.transform(X_train)
X_test_enc = encoder.transform(X_test)

# Let's inspect the transformed variables
X_train_enc[vars_categorical].head()

,A1,A4,A5,A6,A7,A9,A10,A12,A13
596,0.472222,0.512397,0.512397,0.451613,0.418773,0.785156,0.702830,0.445455,0.464853
303,0.472222,0.512397,0.512397,0.625000,0.418773,0.070485,0.250923,0.452471,0.464853
204,0.438806,0.226087,0.226087,0.500000,0.418773,0.785156,0.702830,0.452471,0.464853
351,0.438806,0.226087,0.226087,0.105263,0.146341,0.070485,0.250923,0.452471,0.464853
118,0.438806,0.512397,0.512397,0.423077,0.418773,0.785156,0.702830,0.445455,0.464853


In [6]:
# The idea with k-means encoding is to reduce the feature space

# Here we want to represent 13 variables with 5 features

kmeans = KMeans(n_clusters=5)

kmeans.fit(X_train_enc[vars_categorical])

KMeans(n_clusters=5)

# Clusters + one-hot encoding

In [7]:
# We have the option to encode the variables into the
# clusters they belong to:

X_train_enc["clusters"] = kmeans.fit_predict(X_train_enc[vars_categorical])
X_test_enc["clusters"] = kmeans.predict(X_test_enc[vars_categorical])

X_train_enc["clusters"].head()

596    2
303    3
204    2
351    1
118    2
Name: clusters, dtype: int32

In [8]:
# We would then drop the original variables

X_train_enc.drop(labels=vars_categorical, axis=1, inplace=True)
X_test_enc.drop(labels=vars_categorical, axis=1, inplace=True)

In [9]:
# And then perform one-hot encoding of the clusters

ohe = OneHotEncoder(variables = "clusters",  drop_last=False, ignore_format=True)

ohe.fit(X_train_enc)

# let's transform train and test sets
X_train_enc = ohe.transform(X_train_enc)
X_test_enc = ohe.transform(X_test_enc)

# The final dataset
X_train_enc.head()

,A2,A3,A8,A11,A14,A15,clusters_2,clusters_3,clusters_1,clusters_0,clusters_4
596,46.08,3.000,2.375,8,396.0,4159,1,0,0,0,0
303,15.92,2.875,0.085,0,120.0,0,0,1,0,0,0
204,36.33,2.125,0.085,1,50.0,1187,1,0,0,0,0
351,22.17,0.585,0.000,0,100.0,0,0,0,1,0,0
118,57.83,7.040,14.000,6,360.0,1332,1,0,0,0,0


# Distance to clusters

In [10]:
# let's transform train and test sets

X_train_enc = encoder.transform(X_train)
X_test_enc = encoder.transform(X_test)

In [11]:
# Alternatively we can encode the categorical
# variables into their distances to each of 
# the clusters

clusters = [f"cluster_{i}" for i in range(5)]

X_train_enc[clusters] = kmeans.transform(X_train_enc[vars_categorical])
X_test_enc[clusters] = kmeans.transform(X_test_enc[vars_categorical])

# The final dataset
X_train_enc[clusters].head()

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4
596,0.464075,0.940776,0.104615,0.848369,0.726038
303,0.737578,0.473868,0.857314,0.220917,0.528106
204,0.558597,0.854256,0.355977,0.956136,0.800452
351,0.923416,0.385301,1.055825,0.614738,0.654696
118,0.465905,0.938803,0.121732,0.847238,0.723156


In [12]:
# We would then drop the original variables

X_train_enc.drop(labels=vars_categorical, axis=1, inplace=True)
X_test_enc.drop(labels=vars_categorical, axis=1, inplace=True)

# The final dataset
X_train_enc.head()

,A2,A3,A8,A11,A14,A15,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4
596,46.08,3.000,2.375,8,396.0,4159,0.464075,0.940776,0.104615,0.848369,0.726038
303,15.92,2.875,0.085,0,120.0,0,0.737578,0.473868,0.857314,0.220917,0.528106
204,36.33,2.125,0.085,1,50.0,1187,0.558597,0.854256,0.355977,0.956136,0.800452
351,22.17,0.585,0.000,0,100.0,0,0.923416,0.385301,1.055825,0.614738,0.654696
118,57.83,7.040,14.000,6,360.0,1332,0.465905,0.938803,0.121732,0.847238,0.723156
